In [9]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np 
import gc
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [88]:
np.bincount(np.random.randint(2, size=5000))

array([2516, 2484])

In [219]:
#pred = np.random.rand(5000)
#target_rand = np.random.randint(2, size=5000)
df = pd.DataFrame({'target': np.random.randint(2, size=5000),'pred' : np.random.rand(5000)})
#df['target'] = np.where(df.pred > 0.6,1,0)

In [220]:
df.head()

,target,pred
0,0,0.301301
1,1,0.917180
2,0,0.992003
3,0,0.885999
4,1,0.395961


In [221]:
def gini(target,prediction):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-221-1afa037c16a7>, line 2)

In [222]:
bin = 10
qnt = np.quantile(df.pred,q=np.linspace(0, 1, bin+1),axis=0)
qnt[0] = -0.001
qnt[bin] = 1
qnt

array([-0.001     ,  0.09626088,  0.19269694,  0.29602803,  0.39674114,
        0.49410188,  0.59351011,  0.70221248,  0.80056704,  0.90347787,
        1.        ])

In [223]:
df['band'] = pd.cut(df['pred'], qnt)

In [225]:
df_ct = pd.crosstab(index=df['band'],columns=df['target'],margins=True, margins_name="Total",\
            rownames=['band'], colnames=['index'])
df_ct.reset_index(inplace=True)
df_ct.rename({1 : 'Event' , 0 : 'NonEvent'},axis = 1,inplace=True)

In [226]:
df_ct

index,band,NonEvent,Event,Total
0,"(-0.001, 0.0963]",247,253,500
1,"(0.0963, 0.193]",255,245,500
2,"(0.193, 0.296]",257,243,500
3,"(0.296, 0.397]",237,263,500
4,"(0.397, 0.494]",235,265,500
5,"(0.494, 0.594]",252,248,500
6,"(0.594, 0.702]",256,244,500
7,"(0.702, 0.801]",251,249,500
8,"(0.801, 0.903]",247,253,500
9,"(0.903, 1.0]",253,247,500


In [227]:
df_ct['cum_nonevent'] = df_ct.NonEvent.cumsum()
df_ct.loc[bin,'cum_nonevent'] = df_ct.loc[bin, 'NonEvent']

df_ct['cum_event'] = df_ct.Event.cumsum()
df_ct.loc[bin,'cum_event'] = df_ct.loc[bin, 'Event']

In [232]:
df_ct['population_%'] = df_ct['Total']/df_ct.loc[bin,'Total']

In [233]:
df_ct['cum_nonevent_%'] = df_ct['NonEvent']/df_ct.loc[bin,'NonEvent']
df_ct['cum_event_%'] = df_ct['Event']/df_ct.loc[bin,'Event']

In [241]:
df_ct['difference'] = df_ct['cum_event_%'] - df_ct['cum_nonevent_%']

In [239]:
df_ct['event_rate'] = df_ct['Event']/df_ct.loc[bin,'Event']

In [254]:
df_ct['gini'] = np.nan

In [255]:
df_ct

index,band,NonEvent,Event,Total,cum_nonevent,cum_event,population_%,cum_nonevent_%,cum_event_%,difference,event_rate,gini
0,"(-0.001, 0.0963]",247,253,500,247,253,0.1,0.099197,0.100797,0.0016,0.100797,NaN
1,"(0.0963, 0.193]",255,245,500,502,498,0.1,0.102410,0.097610,-0.0048,0.097610,NaN
2,"(0.193, 0.296]",257,243,500,759,741,0.1,0.103213,0.096813,-0.0064,0.096813,NaN
3,"(0.296, 0.397]",237,263,500,996,1004,0.1,0.095181,0.104781,0.0096,0.104781,NaN
4,"(0.397, 0.494]",235,265,500,1231,1269,0.1,0.094378,0.105578,0.0112,0.105578,NaN
5,"(0.494, 0.594]",252,248,500,1483,1517,0.1,0.101205,0.098805,-0.0024,0.098805,NaN
6,"(0.594, 0.702]",256,244,500,1739,1761,0.1,0.102811,0.097211,-0.0056,0.097211,NaN
7,"(0.702, 0.801]",251,249,500,1990,2010,0.1,0.100803,0.099203,-0.0016,0.099203,NaN
8,"(0.801, 0.903]",247,253,500,2237,2263,0.1,0.099197,0.100797,0.0016,0.100797,NaN
9,"(0.903, 1.0]",253,247,500,2490,2510,0.1,0.101606,0.098406,-0.0032,0.098406,NaN


In [256]:
for i in range(bin):
    if(i == 0):
        df_ct['gini'][i] = 0
    if(i > 0):
         df_ct['gini'][i] = 1

In [257]:
df_ct

index,band,NonEvent,Event,Total,cum_nonevent,cum_event,population_%,cum_nonevent_%,cum_event_%,difference,event_rate,gini
0,"(-0.001, 0.0963]",247,253,500,247,253,0.1,0.099197,0.100797,0.0016,0.100797,0.0
1,"(0.0963, 0.193]",255,245,500,502,498,0.1,0.102410,0.097610,-0.0048,0.097610,1.0
2,"(0.193, 0.296]",257,243,500,759,741,0.1,0.103213,0.096813,-0.0064,0.096813,1.0
3,"(0.296, 0.397]",237,263,500,996,1004,0.1,0.095181,0.104781,0.0096,0.104781,1.0
4,"(0.397, 0.494]",235,265,500,1231,1269,0.1,0.094378,0.105578,0.0112,0.105578,1.0
5,"(0.494, 0.594]",252,248,500,1483,1517,0.1,0.101205,0.098805,-0.0024,0.098805,1.0
6,"(0.594, 0.702]",256,244,500,1739,1761,0.1,0.102811,0.097211,-0.0056,0.097211,1.0
7,"(0.702, 0.801]",251,249,500,1990,2010,0.1,0.100803,0.099203,-0.0016,0.099203,1.0
8,"(0.801, 0.903]",247,253,500,2237,2263,0.1,0.099197,0.100797,0.0016,0.100797,1.0
9,"(0.903, 1.0]",253,247,500,2490,2510,0.1,0.101606,0.098406,-0.0032,0.098406,1.0


In [259]:
for i in range(bin):
    if(i == 0):
        df_ct['gini'][i] = df_ct['cum_nonevent_%'][i] * df_ct['cum_event_%'][i] * 0.5
    if(i > 0):
         df_ct['gini'][i] = (df_ct['cum_nonevent_%'][i] + df_ct['cum_nonevent_%'][i-1])* (df_ct['cum_event_%'][i] - df_ct['cum_event_%'][i-1])*0.5
                                

In [260]:
df_ct

index,band,NonEvent,Event,Total,cum_nonevent,cum_event,population_%,cum_nonevent_%,cum_event_%,difference,event_rate,gini
0,"(-0.001, 0.0963]",247,253,500,247,253,0.1,0.099197,0.100797,0.0016,0.100797,0.004999
1,"(0.0963, 0.193]",255,245,500,502,498,0.1,0.102410,0.097610,-0.0048,0.097610,-0.000321
2,"(0.193, 0.296]",257,243,500,759,741,0.1,0.103213,0.096813,-0.0064,0.096813,-0.000082
3,"(0.296, 0.397]",237,263,500,996,1004,0.1,0.095181,0.104781,0.0096,0.104781,0.000790
4,"(0.397, 0.494]",235,265,500,1231,1269,0.1,0.094378,0.105578,0.0112,0.105578,0.000076
5,"(0.494, 0.594]",252,248,500,1483,1517,0.1,0.101205,0.098805,-0.0024,0.098805,-0.000662
6,"(0.594, 0.702]",256,244,500,1739,1761,0.1,0.102811,0.097211,-0.0056,0.097211,-0.000163
7,"(0.702, 0.801]",251,249,500,1990,2010,0.1,0.100803,0.099203,-0.0016,0.099203,0.000203
8,"(0.801, 0.903]",247,253,500,2237,2263,0.1,0.099197,0.100797,0.0016,0.100797,0.000159
9,"(0.903, 1.0]",253,247,500,2490,2510,0.1,0.101606,0.098406,-0.0032,0.098406,-0.000240


In [240]:
df_ct

index,band,NonEvent,Event,Total,cum_nonevent,cum_event,population_%,cum_nonevent_%,cum_event_%,difference,event_rate
0,"(-0.001, 0.0963]",247,253,500,247,253,0.1,0.099197,0.100797,-0.0016,0.100797
1,"(0.0963, 0.193]",255,245,500,502,498,0.1,0.102410,0.097610,0.0048,0.097610
2,"(0.193, 0.296]",257,243,500,759,741,0.1,0.103213,0.096813,0.0064,0.096813
3,"(0.296, 0.397]",237,263,500,996,1004,0.1,0.095181,0.104781,-0.0096,0.104781
4,"(0.397, 0.494]",235,265,500,1231,1269,0.1,0.094378,0.105578,-0.0112,0.105578
5,"(0.494, 0.594]",252,248,500,1483,1517,0.1,0.101205,0.098805,0.0024,0.098805
6,"(0.594, 0.702]",256,244,500,1739,1761,0.1,0.102811,0.097211,0.0056,0.097211
7,"(0.702, 0.801]",251,249,500,1990,2010,0.1,0.100803,0.099203,0.0016,0.099203
8,"(0.801, 0.903]",247,253,500,2237,2263,0.1,0.099197,0.100797,-0.0016,0.100797
9,"(0.903, 1.0]",253,247,500,2490,2510,0.1,0.101606,0.098406,0.0032,0.098406


In [ ]:
qnt = np.quantile(train_output.Good_bad_flag12_prob,q=np.linspace(0, 1, 11),axis=0)
qnt[0] = 0
qnt[10] = 1
train_output['band'] = pd.cut(train_output['Good_bad_flag12_prob'], qnt)
pd.crosstab(index=train_output['band'],columns=train_output['Good_bad_flag12'],dropna=False)